In [1]:
include("../../MCVI_Julia/src/Planner.jl")
using Plots
using RockSample

## 1. First prepare the POMDP and some basic parameters

In [2]:
pomdp = RockSamplePOMDP(7,8)

action_space = actions(pomdp)
obs_space = observations(pomdp)

nb_particles_b0 = 500
max_node_size = 10000

b0 = initialstate(pomdp)
b0_particles = []
for i in 1:nb_particles_b0
    push!(b0_particles, rand(b0))
end

## 2. Prepare $V_{mdp}$ heuristic

In [ ]:
# define Q learning
Q_table = Dict{Any, Dict{Int64, Float64}}()
learning_rate = 0.9
explore_rate = 0.65
nb_particles_b0 = 10000

b0_particles = []
for i in 1:nb_particles_b0
    push!(b0_particles, rand(b0))
end
Q_learning_policy = Qlearning(Q_table, learning_rate, explore_rate, action_space)
RL = FindRLower(pomdp, b0, action_space)

nb_episode_size = 5
nb_max_episode = 5
nb_sim = 20
epsilon_Q_learning = 0.01
Training(Q_learning_policy, nb_episode_size, nb_max_episode, nb_sim, epsilon_Q_learning, b0_particles, pomdp)

## 3. Create initial FSC and belief Tree Node

In [ ]:
a, U = EvaluateUpperBound(b0_particles, Q_learning_policy)
root_b_tree_node = BeliefTreeNode(b0_particles, Dict{Pair{Any, Any}, BeliefTreeNode}(), a, Dict{Any, Float64}(),U, RL, -1)
fsc = InitFSC(max_node_size, action_space, obs_space)

## 4. MCVI planning

In [ ]:
MCVIPlanning(b0_particles, fsc, pomdp, RL, 20, 100, 0.1, 100, Q_learning_policy, root_b_tree_node)

## 4. Evaluation

In [ ]:
SimulationWithFSC(b0, pomdp, fsc, 40) # Simulate One Time

In [ ]:
EvaluationWithSimulationFSC(b0, pomdp, fsc, discount(pomdp), 10000)